# VisualOdometrySLAM

In [1]:
import random

import cv2
import numpy as np
import pandas as pd

def seed_all(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    cv2.setRNGSeed(seed)

In [2]:
SEED = 42
seed_all(SEED)

In [3]:
from pathlib import Path

DATA_PATH = Path('../data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_PATH_VIDEO = DATA_PATH /Path('visual_odometry/')
DATA_PATH_VIDEO.mkdir(parents=True, exist_ok=True)

DATA_PATH_OUTPUT = DATA_PATH / Path('output_data/')
DATA_PATH_OUTPUT.mkdir(parents=True, exist_ok=True)

DATA_PATH_SAVE_MODELS = DATA_PATH / Path('models/')
DATA_PATH_SAVE_MODELS.mkdir(parents=True, exist_ok=True)

DATA_IMGS = Path('../imgs')
DATA_IMGS.mkdir(parents=True, exist_ok=True)

In [4]:
import sys
import os

project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_path)

## Общие методы

| Параметр                  | Где в YAML                     | Что означает                                  | Как определить / откуда берётся                 |
| ------------------------- | ------------------------------ | --------------------------------------------- | ----------------------------------------------- |
| `fx`, `fy`                | `camera_matrix.data[0]`, `[4]` | Фокусное расстояние по осям X и Y             | Из калибровки (например, `cv2.calibrateCamera`) |
| `cx`, `cy`                | `camera_matrix.data[2]`, `[5]` | Координаты главной точки (оптический центр)   | Центр кадра или из калибровки                   |
| `camera_matrix`           | `camera_matrix.data`           | Внутренняя матрица 3×3                        | Результат калибровки                            |
| `distortion_model`        | `distortion_model`             | Модель дисторсии (`plumb_bob`, `equidistant`) | Выбирается при калибровке, по типу объектива    |
| `distortion_coefficients` | `distortion_coefficients.data` | Коэффициенты искажений (k1, k2, p1, p2, k3)   | Выход `cv2.calibrateCamera`, `Kalibr`, ROS      |
| `image_width` / `height`  | `image_width`, `image_height`  | Размер изображения (в пикселях)               | Известно по сенсору или из примера калибровки   |
| `camera_name`             | `camera_name`                  | Название камеры                               | Задаётся вручную (опционально)                  |


In [5]:
import cv2
import numpy as np
from pathlib import Path
from typing import Union, List


def load_video_frames(video_path: Union[str, Path], target_fps: float) -> List[np.ndarray]:
    """
    Извлекает кадры из видео с заданной частотой и переводит их в оттенки серого.

    :param video_path: путь к видеофайлу
    :param target_fps: целевая частота обработки кадров
    :return: список кадров (в оттенках серого)
    """
    video_path = Path(video_path)
    if not video_path.exists():
        raise FileNotFoundError(f'Видео не найдено по пути: {video_path.resolve()}')

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise IOError(f'Не удалось открыть видеофайл: {video_path}')

    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(video_fps / target_fps)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_sec = total_frames / video_fps
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"Информация о видео:")
    print(f"- Имя файла: {video_path.name}")
    print(f"- Размер кадра: {frame_width} x {frame_height}")
    print(f"- Оригинальный FPS: {video_fps:.2f}")
    print(f"- Общее число кадров: {total_frames}")
    print(f"- Длительность видео: {duration_sec:.2f} секунд")
    print(f"- Целевая частота обработки: {target_fps} FPS")
    print(f"- Будет обрабатываться каждый {frame_interval}-й кадр")

    processed_frames = []
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_interval == 0:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            processed_frames.append(gray)

        frame_idx += 1

    cap.release()
    print(f"Обработано кадров: {len(processed_frames)} при целевом FPS: {target_fps}")
    return processed_frames


def load_kitti_sequence(path: Union[str, Path], grayscale: bool = True) -> List[np.ndarray]:
    image_paths = sorted(Path(path).glob("*.png"))
    frames = []
    for p in image_paths:
        img = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE if grayscale else cv2.IMREAD_COLOR)
        if img is not None:
            frames.append(img)
    return frames


## ORB params

| Параметр        | Тип     | Значение по умолчанию  | Описание и рекомендации                                                                                                         |
| --------------- | ------- | ---------------------- | ------------------------------------------------------------------------------------------------------------------------------- |
| `nfeatures`     | `int`   | 500                    | Максимальное количество фич, которые ORB попытается найти. <br>🔧 Увеличь до 3000–5000 для богатой сцены или сеточной детекции. |
| `scaleFactor`   | `float` | 1.2                    | Масштаб между уровнями пирамиды. <br>Меньше — больше перекрытий, лучше для масштабных сцен, но медленнее.                       |
| `nlevels`       | `int`   | 8                      | Кол-во уровней в пирамиде изображений. <br>🔧 Увеличь до 12–16, если ожидается масштабный диапазон объектов.                    |
| `edgeThreshold` | `int`   | 31                     | Отступ от границ изображения. <br>🔧 Уменьши до 10–15, если хочешь находить фичи у краёв.                                       |
| `firstLevel`    | `int`   | 0                      | Начальный уровень пирамиды (обычно 0). Почти никогда не меняется.                                                               |
| `WTA_K`         | `int`   | 2                      | Метод генерации BRIEF-дескриптора: 2 = быстрее, 3 или 4 = точнее. <br>Используй 2, если нет проблем с точностью.                |
| `scoreType`     | `int`   | `cv2.ORB_HARRIS_SCORE` | Выбор способа ранжирования углов: `cv2.ORB_HARRIS_SCORE` или `cv2.ORB_FAST_SCORE`. <br>`HARRIS` точнее, `FAST` — быстрее.       |
| `patchSize`     | `int`   | 31                     | Размер области вокруг фичи для дескриптора. <br>🔧 31 — стандарт, можно увеличить до 49, если объект крупный.                   |
| `fastThreshold` | `int`   | 20                     | Порог FAST-угла: чем меньше — тем больше фич. <br>🔧 Снизь до 5–10, если ORB пропускает фичи.                                   |


In [6]:
import yaml
from pathlib import Path
from collections import Counter
from typing import Union, List, Optional

import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
class MapPoint:
    def __init__(self, coord_3d: np.ndarray, descriptor: np.ndarray):
        self.coord = coord_3d  # (X, Y, Z)
        self.descriptor = descriptor
        self.observations = []  # список (frame_idx, keypoint_idx, (x, y))


class KeyFrame:
    def __init__(self, frame_id: int, image: np.ndarray, keypoints, descriptors, pose: np.ndarray):
        self.id = frame_id
        self.image = image
        self.keypoints = keypoints
        self.descriptors = descriptors
        self.pose = pose        

In [ ]:
from visual_slam.config import Config
from visual_slam.slam import SlamBase
from visual_slam.camera.camera import PinholeCamera
from visual_slam.source import DatasetSource, CameraSource

class VisualOdometrySLAM(SlamBase):

    def __init__(self, config: Config):
        super().__init__()
        self.config = config

        if config.dataset_path is not None:
            self.source = DatasetSource(config.dataset_path)
        else:
            self.source = CameraSource(
                camera_id=config.camera_id,
                width=config.width,
                height=config.height,
                fps=config.fps
            )

        # Камера
        self.camera = PinholeCamera(
            width=config.width,
            height=config.height,
            fx=config.fx,
            fy=config.fy,
            cx=config.cx,
            cy=config.cy,
            dist_coeffs=config.dist_coeffs,
            fps=config.fps,
            bf=config.bf
        )

        # ORB детектор и матчинг
        self.orb = cv2.ORB_create(
            nfeatures=config.orb_nfeatures,
            scaleFactor=config.orb_scale_factor,
            nlevels=config.orb_nlevels,
            edgeThreshold=config.orb_edge_threshold,
            fastThreshold=config.orb_fast_threshold
        )
        self.matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

        # Служебные контейнеры
        self.keyframes: List = []
        self.mappoints: List = []
        self.poses: List[np.ndarray] = []

    # -----------------------------
    # Основной цикл
    # -----------------------------
    def run(self):
        """Запуск процесса визуальной одометрии и SLAM."""
        print("▶ Запуск визуальной одометрии...")

        while self.source.is_ok():
            frame, timestamp = self.source.get_frame()
            if frame is None:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            keypoints, descriptors = self.orb.detectAndCompute(gray, None)

            # TODO: обработка фич, матчинг, PnP, триангуляция и обновление карты
            print(f"[{timestamp:.3f}] Ключевых точек: {len(keypoints)}")

        print("✔ Завершено")

    def reset(self):
        """Сброс состояния (например, при повторном запуске)."""
        self.keyframes.clear()
        self.mappoints.clear()
        self.poses.clear()
        self.source.reset()

In [9]:
import math
import time
from collections import Counter
from typing import List, Tuple

import numpy as np
import cv2
import g2o


class MatcherFilterMixin:
    @staticmethod
    def filter_by_ratio_test(knn_matches, kp1, kp2, ratio_thresh):
        good_matches = []
        pts1, pts2 = [], []

        for pair in knn_matches:
            if len(pair) < 2:
                continue
            m, n = pair
            if m.distance < ratio_thresh * n.distance:
                good_matches.append(m)
                pts1.append(kp1[m.queryIdx].pt)
                pts2.append(kp2[m.trainIdx].pt)

        return good_matches, np.float32(pts1), np.float32(pts2)

    @staticmethod
    def filter_by_max_distance(matches, pts1, pts2, max_dist):
        filtered_matches = []
        filtered_pts1, filtered_pts2 = [], []

        for i, m in enumerate(matches):
            if m.distance < max_dist:
                filtered_matches.append(m)
                filtered_pts1.append(pts1[i])
                filtered_pts2.append(pts2[i])

        return filtered_matches, np.float32(filtered_pts1), np.float32(filtered_pts2)

    @staticmethod
    def filter_unique_matches(matches, pts1, pts2):
        seen_trainIdx = set()
        unique_matches = []
        unique_pts1, unique_pts2 = [], []

        for i, m in enumerate(matches):
            if m.trainIdx not in seen_trainIdx:
                seen_trainIdx.add(m.trainIdx)
                unique_matches.append(m)
                unique_pts1.append(pts1[i])
                unique_pts2.append(pts2[i])

        return unique_matches, np.float32(unique_pts1), np.float32(unique_pts2)

    @staticmethod
    def filter_by_ransac_fundamental(matches, pts1, pts2):
        if len(matches) < 8:
            return [], np.array([]), np.array([])

        F, mask = cv2.findFundamentalMat(pts1, pts2, method=cv2.FM_RANSAC, ransacReprojThreshold=1.0)

        inlier_matches = []
        inlier_pts1, inlier_pts2 = [], []

        if mask is not None:
            for i, m in enumerate(matches):
                if mask[i]:
                    inlier_matches.append(m)
                    inlier_pts1.append(pts1[i])
                    inlier_pts2.append(pts2[i])

        return inlier_matches, np.float32(inlier_pts1), np.float32(inlier_pts2)

    @staticmethod
    def filter_by_exclusion_mask(matches, pts1, pts2, mask_exclude_regions: List[Tuple[int, int, int, int]]):
        if not mask_exclude_regions:
            return matches, pts1, pts2

        filtered_matches = []
        filtered_pts1 = []
        filtered_pts2 = []

        for i, m in enumerate(matches):
            x1, y1 = pts1[i]
            x2, y2 = pts2[i]

            excluded = any(
                (xmin <= x1 <= xmax and ymin <= y1 <= ymax) or
                (xmin <= x2 <= xmax and ymin <= y2 <= ymax)
                for (xmin, ymin, xmax, ymax) in mask_exclude_regions
            )

            if not excluded:
                filtered_matches.append(m)
                filtered_pts1.append((x1, y1))
                filtered_pts2.append((x2, y2))

        return filtered_matches, np.float32(filtered_pts1), np.float32(filtered_pts2)

    @staticmethod
    def filter_mutual_matches(
        matcher: cv2.DescriptorMatcher,
        des1: np.ndarray,
        des2: np.ndarray,
        kp1: List[cv2.KeyPoint],
        kp2: List[cv2.KeyPoint],
        matches: List[cv2.DMatch],
        pts1: np.ndarray,
        pts2: np.ndarray
    ) -> Tuple[List[cv2.DMatch], np.ndarray, np.ndarray]:
        """
        Оставляет только взаимные совпадения (A→B и B→A).
        
        :param matcher: объект cv2.DescriptorMatcher (например, BFMatcher или FlannBasedMatcher)
        :param des1: дескрипторы первого изображения
        :param des2: дескрипторы второго изображения
        :param kp1: ключевые точки первого изображения
        :param kp2: ключевые точки второго изображения
        :param matches: прямые совпадения A→B
        :param pts1: координаты точек первого изображения
        :param pts2: координаты точек второго изображения
        :return: (взаимные совпадения, pts1, pts2)
        """
        reverse_matches = matcher.match(des2, des1)
        reverse_set = {(m.trainIdx, m.queryIdx) for m in reverse_matches}

        mutual_matches = []
        mutual_pts1, mutual_pts2 = [], []

        for i, m in enumerate(matches):
            if (m.queryIdx, m.trainIdx) in reverse_set:
                mutual_matches.append(m)
                mutual_pts1.append(pts1[i])
                mutual_pts2.append(pts2[i])

        return mutual_matches, np.float32(mutual_pts1), np.float32(mutual_pts2)

In [11]:
import cv2
from pathlib import Path

VIDEO_PATH = DATA_PATH / 'KITTI_sequence_1/image_l'
CALIBRATION_PATH = DATA_PATH / 'KITTI_sequence_1/calib.txt'
TARGET_FPS = 2.0

if not VIDEO_PATH.exists():
    raise FileNotFoundError(f'Видео не найдено по пути: {VIDEO_PATH.resolve()}')

if not CALIBRATION_PATH.exists():
    raise FileNotFoundError(f'Калибровочный файл не найден: {CALIBRATION_PATH.resolve()}')

params = {
    "target_fps": TARGET_FPS,  # частота обработки кадров

    # ORB-параметры
    "nfeatures": 100,             # [default=500] больше фич — выше плотность покрытия сцены
    "fastThreshold": 10,          # [default=20] ниже порог — чувствительнее к слабым углам
    "edgeThreshold": 20,           # [default=31] позволяет детектировать ближе к краям кадра
    "scaleFactor": 1.2,           # [default=1.2] масштаб между уровнями пирамиды (меньше — медленнее, но точнее)
    "nlevels": 8,                 # [default=8] число уровней пирамиды — выше = лучше устойчивость к масштабу

    # Фильтрация матчей по дистанции (Lowe’s ratio test)
    "ratio_thresh": 0.75,          # [default=0.75] меньше строгая фильтрация
    "max_match_distance": 200,    # максимальная допустимая дистанция матча
    # "use_mutual_check": True,      # включить/выключить взаимную проверку

    # Grid-детектор (разделение кадра на сетку)
    "use_grid": False,             # включение/выключение сетки
    "grid_rows": 4,
    "grid_cols": 6,

    # ANMS (равномерное распределение фичей)
    "use_anms": True,             # включить/выключить ANMS после ORB
    "anms_count": 3000,            # сколько фич оставить после ANMS (подбери под свою сцену)
    "anms_tolerance": 0.2,  # порог ANMS (чем меньше, тем более равномерно распределены фичи)

    # FLANN-матчер (быстрый поиск похожих фичей)
    "flann_table_number": 10,
    "flann_key_size": 14,
    "flann_multi_probe_level": 2,
    "flann_checks": 50,
    
    "init_keyframes":10,         # минимальное число KeyFrame для инициализации карты
    "last_frame_size":10,         # количество последних кадров для g2o Bundle Adjustment
    "g2o_optimize_step": 200,  # шаг оптимизации g2o (количество итераций)
    "ba_obs_size": 2,            # минимальное число наблюдений для Bundle Adjustment

    # Добавь сюда другие нужные параметры (например, для keyframe/track decision)
    "min_tracked_points": 30,      # минимальное число трекаемых MapPoints для удержания трекинга
    "keyframe_interval": 3,        # минимальное число кадров между KeyFrame
    # "keyframe_translation_thresh": 0.2, # минимальное смещение камеры для нового KeyFrame
}

vo = VisualOdometry(
    video_path=VIDEO_PATH,
    calibration_file=CALIBRATION_PATH,
    **params
)

[KITTI] Загружено 51 кадров из image_l


In [ ]:
# from IPython.display import display, clear_output
# from PIL import Image
# import time
# import cv2

# num_frames_to_show = int(TARGET_FPS * 30)
# frames_to_show = vo.frames[:num_frames_to_show]

# for frame in frames_to_show:
#     rgb_frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
#     clear_output(wait=True)
#     display(Image.fromarray(rgb_frame))
#     time.sleep(1 / TARGET_FPS / 2)

In [13]:
vo.clear_data()
vo.run()

[Init] Запуск инициализации карты...
[Debug] Найдено 8105 ключевых точек
[Debug] После ANMS осталось 2763 точек
[Init] Первый кадр: #0, найдено 2763 ключевых точек.
[Init:1] Обработка кадра #1..............
[Debug] Найдено 8196 ключевых точек
[Debug] После ANMS осталось 2860 точек
[Init:1] Сопоставлено 676 фич между кадрами 0 и 1
[Init:1] Средний параллакс: 15.42
[Pose:1] Камера на позиции: X=0.013, Y=-0.021, Z=1.000
[Init:1] Добавлен KeyFrame #1
[Triangulate] Всего: 676 | валидных Z∈(0.01,200.0): 642
[Init:1] Триангуляция дала 642 3D-точек.
[Init:1] Добавлено 642 новых 3D-точек после фильрации.
[Init:1] Добавлено 0 новых наблюдений к существующим MapPoints.
[Init:1] KeyFrames: 2 | MapPoints: 642
[Init:2] Обработка кадра #2..............
[Debug] Найдено 8286 ключевых точек
[Debug] После ANMS осталось 2850 точек
[Init:2] Сопоставлено 562 фич между кадрами 1 и 2
[Init:2] Средний параллакс: 18.34
[Pose:2] Камера на позиции: X=0.009, Y=-0.054, Z=1.999
[Init:2] Добавлен KeyFrame #2
[Triangu

In [14]:
# vo.visualize_matches()

In [15]:
import open3d as o3d
import numpy as np

def create_camera_frustum(scale: float = 0.05) -> o3d.geometry.LineSet:
    """
    Создаёт 3D-модель пирамиды камеры для визуализации в open3d.
    scale – масштаб фрустума
    """
    points = np.array([
        [0, 0, 0],
        [ scale,  scale,  scale * 2],
        [ scale, -scale,  scale * 2],
        [-scale, -scale,  scale * 2],
        [-scale,  scale,  scale * 2],
    ])
    lines = [
        [0, 1], [0, 2], [0, 3], [0, 4],
        [1, 2], [2, 3], [3, 4], [4, 1]
    ]
    frustum = o3d.geometry.LineSet()
    frustum.points = o3d.utility.Vector3dVector(points)
    frustum.lines = o3d.utility.Vector2iVector(lines)
    frustum.paint_uniform_color([0.0, 0.6, 1.0])
    return frustum


def check_and_visualize_mappoints(vo):
    """
    Проверяет и визуализирует 3D MapPoints после инициализации.
    vo: экземпляр класса VisualOdometry
    """
    if not vo.mappoints:
        print("[Check] Нет MapPoints для отображения.")
        return

    coords = np.array([mp.coord for mp in vo.mappoints if np.all(np.isfinite(mp.coord))])

    if coords.size == 0:
        print("[Check] Все точки невалидны.")
        return

    # Статистика
    depths = coords[:, 2]
    print(f"[Check] Всего точек: {len(coords)}")
    print(f"[Check] Глубина Z: mean={depths.mean():.3f}, median={np.median(depths):.3f}, min={depths.min():.3f}, max={depths.max():.3f}")

    print(f"[Check] X: mean={coords[:,0].mean():.3f}, min={coords[:,0].min():.3f}, max={coords[:,0].max():.3f}")
    print(f"[Check] Y: mean={coords[:,1].mean():.3f}, min={coords[:,1].min():.3f}, max={coords[:,1].max():.3f}")

    print("[Check] Примеры точек:")
    for i in range(min(5, len(coords))):
        print(f"   {coords[i]}")

    # Создаём PointCloud для Open3D
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(coords)
    pcd.paint_uniform_color([1.0, 0.0, 0.0])  # красные точки

    # Добавляем камеры (если есть позы)
    geometries = [pcd]
    for pose in vo.poses:
        frustum = create_camera_frustum(scale=5).transform(pose)
        geometries.append(frustum)

    # Визуализация
    o3d.visualization.draw_geometries(geometries, window_name="MapPoints после инициализации")

# check_and_visualize_mappoints(vo)